In [1]:
import pandas as pd
import requests
import re
import cufflinks as cf
from datetime import datetime
import plotly.io as pio

cf.go_offline(connected=True)

URL = "https://livecorona.co.kr/data/koreaRegionalCumulativeData.js"
response = requests.get(URL)
text = re.sub("[\n\s]", "", response.text)
text = text[text.find("=") + 3 : len(text) - 3]
# 후방탐색
text = re.split("(?<=\])\,", text)
data = list()
for index in range(0, len(text)):
    text[index] = re.sub('[" \[ \] ]', "", text[index])
    data.append(list(text[index].split(",")))
data

[['2/1', '0', '1', '12', '0'],
 ['2/2', '0', '0', '15', '0'],
 ['2/3', '0', '0', '15', '0'],
 ['2/4', '0', '0', '16', '0'],
 ['2/5', '0', '0', '19', '0'],
 ['2/6', '0', '0', '23', '0'],
 ['2/7', '0', '0', '24', '1'],
 ['2/8', '0', '0', '24', '1'],
 ['2/9', '0', '0', '25', '3'],
 ['2/10', '0', '0', '27', '3'],
 ['2/11', '0', '0', '28', '3'],
 ['2/12', '0', '0', '28', '7'],
 ['2/13', '0', '0', '28', '7'],
 ['2/14', '0', '0', '28', '7'],
 ['2/15', '0', '0', '28', '9'],
 ['2/16', '0', '0', '29', '9'],
 ['2/17', '0', '0', '30', '10'],
 ['2/18', '0', '0', '31', '12'],
 ['2/19', '0', '0', '31', '12'],
 ['2/20', '1', '1', '104', '16'],
 ['2/21', '2', '1', '204', '16'],
 ['2/22', '2', '0', '433', '16'],
 ['2/23', '6', '4', '602', '18'],
 ['2/24', '8', '2', '833', '18'],
 ['2/25', '10', '2', '977', '22'],
 ['2/26', '12', '2', '1261', '22'],
 ['2/27', '13', '1', '1766', '22'],
 ['2/28', '13', '0', '2337', '22'],
 ['2/29', '16', '3', '3150', '27'],
 ['3/1', '17', '1', '3736', '30'],
 ['3/2', '28',

In [2]:
# Data 정제 부분
# raw Data to Dataframe
data_dict = {data[i][0]: data[i][1:] for i in range(len(data))}
df = pd.DataFrame(data_dict, index=["사망자", "확진자 증가수", "확진자", "완치자"])
df = df.transpose()
for col in df.keys():
    df[col] = df[col].apply(lambda val: int(val))
cal_list = list()
cal_list.append(df["확진자"][0])
for index in range(1, len(df["확진자"])):
    cal_list.append(df["확진자"][index] - df["확진자"][index - 1])
df["확진자 증가수"] = cal_list

# change columns order
df = df[["확진자", "확진자 증가수", "완치자", "사망자"]]
df

,확진자,확진자 증가수,완치자,사망자
2/1,12,12,0,0
2/2,15,3,0,0
2/3,15,0,0,0
2/4,16,1,0,0
2/5,19,3,0,0
...,...,...,...,...
8/20,16670,324,14120,309
8/21,17002,332,14169,309
8/22,17399,397,14200,309
8/23,17665,266,14219,309


In [3]:
pio.renderers
year = datetime.today().year
month = datetime.today().month
day = datetime.today().day
title = f"COVID-19 현황 [날짜 기준 : {year}.{month}.{day}.00시 기준]"

In [4]:
fig = df.iplot(
    subplots=True,
    shape=(4, 1),
    shared_xaxes=True,
    asFigure=True,
    kind="line",
    title=title,
    subplot_titles=True,
    legend=False,
    fill=True,
)
fig.show()
pio.write_html(fig, file=".\COVID.html", auto_open=True)
# change to .png image
# fig.write_image("test.png")
